In [12]:
import numpy as np
from MyGenerator import DataGenerator

In [13]:
import tensorflow as tf
#from tensorflow import keras
import keras
from keras.callbacks import ModelCheckpoint

In [14]:
import pickle
import os

In [15]:
with open(os.path.join("classifier_data", 'labels.pickle'), 'rb') as handle:
    labels = pickle.load(handle)

In [16]:
len(labels)

30000

In [17]:
list_of_labels = [*labels]
np.random.shuffle(list_of_labels)

In [18]:
partition = {'train': list_of_labels[:24000], 
             'validation': list_of_labels[24000:27000],
             'test': list_of_labels[27000:]}

In [19]:
params = {'dim': (2500,),
          'batch_size': 64,
          'n_classes': 3,
          'n_channels': 1,
          'shuffle': True}
test_params = {'dim': (2500,),
               'batch_size': len(partition['test']),
               'n_classes': 3,
               'n_channels': 1,
               'shuffle': True}

In [20]:
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)
testing_generator = DataGenerator(partition['test'], labels, **test_params)

In [21]:
inputs = keras.Input(shape = (2500,1))
conv_1_1 = keras.layers.Conv1D(32, kernel_size = 3, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(inputs)
conv_1_2 = keras.layers.Conv1D(32, kernel_size = 3, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(conv_1_1)
conv_1_3 = keras.layers.Conv1D(32, kernel_size = 3, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(conv_1_2)
pool_1 = keras.layers.MaxPool1D(pool_size = 4)(conv_1_3)

conv_2_1 = keras.layers.Conv1D(64, kernel_size = 5, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(pool_1)
conv_2_2 = keras.layers.Conv1D(64, kernel_size = 5, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(conv_2_1)
#conv_2_3 = keras.layers.Conv1D(64, kernel_size = 3, activation = 'relu', 
#                            kernel_initializer = keras.initializers.he_normal())(conv_2_2)
pool_2 = keras.layers.MaxPool1D(pool_size = 4)(conv_2_2)

conv_3_1 = keras.layers.Conv1D(128, kernel_size = 7, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(pool_2)
#conv_3_2 = keras.layers.Conv1D(128, kernel_size = 3, activation = 'relu', 
#                            kernel_initializer = keras.initializers.he_normal())(conv_3_1)
#conv_3_3 = keras.layers.Conv1D(128, kernel_size = 3, activation = 'relu', 
#                            kernel_initializer = keras.initializers.he_normal())(conv_3_2)
pool_3 = keras.layers.MaxPool1D(pool_size = 4)(conv_3_1)



flat_1 = keras.layers.Flatten()(pool_3)



dense_1 = keras.layers.Dense(1024, activation = 'relu', 
                             kernel_initializer = keras.initializers.he_normal())(flat_1)
outputs = keras.layers.Dense(3, activation = 'softmax')(dense_1)
model = keras.Model(inputs = inputs, outputs = outputs)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2500, 1)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 2498, 32)          128       
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 2496, 32)          3104      
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 2494, 32)          3104      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 623, 32)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 619, 64)           10304     
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 615, 64)           20544     
__________

In [24]:
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.02, amsgrad=False)
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.cls.hdf5', 
                               verbose=2, save_best_only=True)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit_generator(generator=training_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=training_generator.__len__(),
                              #workers=6,
                              callbacks=[checkpointer],
                              epochs = 100,
                              use_multiprocessing=True)

Epoch 1/100
375/375 [==============================] - 13s 35ms/step - loss: 0.1498 - acc: 0.9547 - val_loss: 0.1161 - val_acc: 0.9657

Epoch 00001: val_loss improved from inf to 0.11607, saving model to saved_models/weights.best.cls.hdf5
Epoch 2/100
375/375 [==============================] - 11s 29ms/step - loss: 0.0790 - acc: 0.9769 - val_loss: 0.1158 - val_acc: 0.9684

Epoch 00002: val_loss improved from 0.11607 to 0.11579, saving model to saved_models/weights.best.cls.hdf5
Epoch 3/100
375/375 [==============================] - 11s 30ms/step - loss: 0.0671 - acc: 0.9803 - val_loss: 0.0926 - val_acc: 0.9721

Epoch 00003: val_loss improved from 0.11579 to 0.09263, saving model to saved_models/weights.best.cls.hdf5
Epoch 4/100
375/375 [==============================] - 11s 30ms/step - loss: 0.0587 - acc: 0.9828 - val_loss: 0.0883 - val_acc: 0.9745

Epoch 00004: val_loss improved from 0.09263 to 0.08833, saving model to saved_models/weights.best.cls.hdf5
Epoch 5/100
375/375 [===========

375/375 [==============================] - 12s 31ms/step - loss: 0.0236 - acc: 0.9943 - val_loss: 0.0824 - val_acc: 0.9789

Epoch 00040: val_loss did not improve from 0.07338
Epoch 41/100
375/375 [==============================] - 11s 30ms/step - loss: 0.0234 - acc: 0.9942 - val_loss: 0.0803 - val_acc: 0.9783

Epoch 00041: val_loss did not improve from 0.07338
Epoch 42/100
375/375 [==============================] - 11s 30ms/step - loss: 0.0233 - acc: 0.9944 - val_loss: 0.0780 - val_acc: 0.9793

Epoch 00042: val_loss did not improve from 0.07338
Epoch 43/100
375/375 [==============================] - 12s 31ms/step - loss: 0.0229 - acc: 0.9945 - val_loss: 0.0805 - val_acc: 0.9783

Epoch 00043: val_loss did not improve from 0.07338
Epoch 44/100
375/375 [==============================] - 11s 30ms/step - loss: 0.0227 - acc: 0.9943 - val_loss: 0.0799 - val_acc: 0.9783

Epoch 00044: val_loss did not improve from 0.07338
Epoch 45/100
375/375 [==============================] - 11s 30ms/step - l


Epoch 00083: val_loss did not improve from 0.07338
Epoch 84/100
375/375 [==============================] - 11s 30ms/step - loss: 0.0184 - acc: 0.9959 - val_loss: 0.0768 - val_acc: 0.9793

Epoch 00084: val_loss did not improve from 0.07338
Epoch 85/100
375/375 [==============================] - 11s 30ms/step - loss: 0.0182 - acc: 0.9957 - val_loss: 0.0836 - val_acc: 0.9779

Epoch 00085: val_loss did not improve from 0.07338
Epoch 86/100
375/375 [==============================] - 11s 29ms/step - loss: 0.0181 - acc: 0.9957 - val_loss: 0.0800 - val_acc: 0.9789

Epoch 00086: val_loss did not improve from 0.07338
Epoch 87/100
375/375 [==============================] - 11s 29ms/step - loss: 0.0181 - acc: 0.9958 - val_loss: 0.0797 - val_acc: 0.9793

Epoch 00087: val_loss did not improve from 0.07338
Epoch 88/100
375/375 [==============================] - 11s 29ms/step - loss: 0.0181 - acc: 0.9957 - val_loss: 0.0780 - val_acc: 0.9803

Epoch 00088: val_loss did not improve from 0.07338
Epoch 89

In [25]:
model.load_weights('saved_models/weights.best.cls.hdf5')
model.evaluate_generator(testing_generator)

[0.0692937895655632, 0.9829999804496765]

In [26]:
model.save('saved_models/classifier_model.h5')